In [1]:
import pandas as pd
import numpy as np
import os

os.chdir("..")

In [6]:
# Read in datasets
members_df = pd.read_csv("artifacts/data_ingestion/members_v3.csv")
transactions_df = pd.read_csv("artifacts/data_ingestion/transactions_v2.csv")
user_logs_df = pd.read_csv("artifacts/data_ingestion/user_logs_v2.csv")
train_df = pd.read_csv("artifacts/data_ingestion/train_v2.csv")

### Members Dataset:
- Drop ages and gender due to outliers
- Drop registration time

In [3]:
members_df_drop = members_df.drop(["bd","gender","registration_init_time"], axis=1)

### User Logs Dataset
- Sum features by user IDs

In [4]:
user_logs_df_summed = user_logs_df.groupby("msno").sum()
user_logs_df_summed.reset_index(inplace=True)

### Transactions
- Get latest transaction of each user ID
- Add feature num_prev_transactions
- Add feature total_prev_paid
- Add feature num_prev_cancelled
- Add feature num_prev_discounts
- Add feature curr_discount
- Drop membership expiry date


In [5]:
latest_transactions = transactions_df.copy()
latest_transactions = latest_transactions.sort_values('transaction_date').drop_duplicates(['msno'], keep='last')

In [6]:
prev_transactions = transactions_df.copy()
mask = prev_transactions.apply(tuple, axis=1).isin(latest_transactions.apply(tuple, axis=1))
prev_transactions = prev_transactions[~mask]

In [7]:
num_transactions = prev_transactions.groupby('msno').size().reset_index(name='num_prev_transactions')
total_prev_paid = prev_transactions.groupby('msno')['actual_amount_paid'].sum().reset_index(name='total_prev_paid')
total_prev_cancelled = prev_transactions.groupby('msno')['is_cancel'].sum().reset_index(name='total_prev_cancelled')
num_prev_discounts = prev_transactions[prev_transactions['plan_list_price'] > prev_transactions['actual_amount_paid']].groupby('msno').size().reset_index(name='num_prev_discounts')

# Add new features
latest_transactions = latest_transactions.merge(num_transactions, on='msno', how='left')
latest_transactions = latest_transactions.merge(total_prev_paid, on='msno', how='left')
latest_transactions = latest_transactions.merge(num_prev_discounts, on='msno', how='left')
latest_transactions = latest_transactions.merge(total_prev_cancelled, on='msno', how='left')

# Filling NA values with 0 (for users with no previous transactions)
latest_transactions['num_prev_transactions'] = latest_transactions['num_prev_transactions'].fillna(0)
latest_transactions['total_prev_paid'] = latest_transactions['total_prev_paid'].fillna(0)
latest_transactions['num_prev_discounts'] = latest_transactions['num_prev_discounts'].fillna(0)
latest_transactions['total_prev_cancelled'] = latest_transactions['total_prev_cancelled'].fillna(0)

latest_transactions["curr_discount"] = (latest_transactions['plan_list_price'] > latest_transactions['actual_amount_paid']).astype(int)
latest_transactions = latest_transactions.drop('membership_expire_date',axis=1)
latest_transactions = latest_transactions.drop('transaction_date',axis=1)

### Merge with train dataset

In [8]:
final_dataset = train_df.merge(latest_transactions, on='msno', how='inner')
final_dataset = final_dataset.merge(user_logs_df_summed, on='msno', how='inner')
final_dataset = final_dataset.merge(members_df_drop, on='msno', how='inner')

In [9]:
final_dataset = final_dataset.drop("msno", axis=1)

In [10]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725722 entries, 0 to 725721
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   is_churn               725722 non-null  int64  
 1   payment_method_id      725722 non-null  int64  
 2   payment_plan_days      725722 non-null  int64  
 3   plan_list_price        725722 non-null  int64  
 4   actual_amount_paid     725722 non-null  int64  
 5   is_auto_renew          725722 non-null  int64  
 6   transaction_date       725722 non-null  int64  
 7   is_cancel              725722 non-null  int64  
 8   num_prev_transactions  725722 non-null  float64
 9   total_prev_paid        725722 non-null  float64
 10  num_prev_discounts     725722 non-null  float64
 11  total_prev_cancelled   725722 non-null  float64
 12  curr_discount          725722 non-null  int32  
 13  date                   725722 non-null  int64  
 14  num_25                 725722 non-nu

: 